# Testing compressed residual size with Huffman
In this notebook, we will look at all of test data and see what kind of compression we can achieve on the binarized residuals

In [113]:
import torch 
import numpy as np
import os
import sys
sys.path.append("..")
from src.model import Encoder, Decoder
from src.dataset import ResidualDataset
from src.HuffmanCompression import HuffmanCoding
import pdb
from copy import copy
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load encoder and decoder

In [72]:
checkpoint_enc = '/Users/aditya/Documents/cs348k/Project/NeuralVideoCompression/checkpoint/testing/encoder-epoch_49_iter_40550.pth'
checkpoint_dec = '/Users/aditya/Documents/cs348k/Project/NeuralVideoCompression/checkpoint/testing/decoder-epoch_49_iter_40550.pth'
enc = Encoder()
dec = Decoder()
# load both models
enc.load_state_dict(torch.load(checkpoint_enc, map_location='cpu'))
dec.load_state_dict(torch.load(checkpoint_dec, map_location='cpu'))

In [73]:
train_data_path = '/Users/aditya/Downloads/Video_data/t'
device = torch.device('cpu')
dSet_train = ResidualDataset(train_data_path, 'test', device)
dataset_train = torch.utils.data.DataLoader(dSet_train,
                                            batch_size=1, shuffle=True,
                                            num_workers=0)

dataset mode: test, length: 21


## Loop to get all the reisduals

In [74]:
bin_out_list = []
with torch.no_grad():
        for idx, sample in enumerate(dataset_train):
            model_input = sample['image']
            bin_out = enc(model_input).squeeze().numpy()
            bin_out_list.append(bin_out)

In [75]:
# check it's shape
bin_out_arr = ((np.array(bin_out_list) + 1)/2).astype(int)
bin_out_arr.shape

(21, 32, 45, 150)

## Loop through the tensor, chunk one dimension, build a dict with chunk frequency

In [108]:
N,H,W,C = bin_out_arr.shape
chunk_size = 30
huffmap ={}
i=j=k=l=0
while i<N:
    j=0
    while j<H:
        k=0
        while k<W:
            channel = np.split(bin_out_arr[i][j][k][:],chunk_size)
            for num in channel:
                s =""
                for ele in num:
                    s += str(ele)
                huffmap[s] = huffmap.get(s, 0) + 1
            k+=1
        j+=1
    i+=1
            

In [109]:
huffmap # print map

{'11111': 132874,
 '11101': 25038,
 '11001': 18611,
 '01111': 34519,
 '10000': 33449,
 '01001': 15128,
 '10110': 14659,
 '11110': 34474,
 '11011': 23747,
 '00010': 25821,
 '00110': 19085,
 '10001': 18046,
 '11100': 27567,
 '00000': 117880,
 '00001': 34678,
 '01011': 15570,
 '11010': 15663,
 '00011': 27672,
 '01100': 18714,
 '01000': 25567,
 '10111': 24391,
 '10100': 15679,
 '11000': 27073,
 '01110': 18128,
 '10011': 18747,
 '00100': 24768,
 '00111': 27454,
 '10101': 12975,
 '00101': 15785,
 '10010': 15180,
 '01010': 13150,
 '01101': 15108}

## Now let's use the huffman function to heapify it

In [110]:
h = HuffmanCoding()
h.make_heap(huffmap)
h.merge_nodes()
h.make_codes()
h.codes

{'11001': '00000',
 '01100': '00001',
 '10011': '00010',
 '00110': '00011',
 '11011': '00100',
 '10111': '00101',
 '00100': '00110',
 '11101': '00111',
 '01000': '01000',
 '00010': '01001',
 '10101': '010100',
 '01010': '010101',
 '11000': '01011',
 '00111': '01100',
 '11100': '01101',
 '00011': '01110',
 '10110': '011110',
 '01101': '011111',
 '00000': '100',
 '01001': '101000',
 '10010': '101001',
 '01011': '101010',
 '11010': '101011',
 '10100': '101100',
 '00101': '101101',
 '10000': '10111',
 '11111': '110',
 '11110': '11100',
 '01111': '11101',
 '00001': '11110',
 '10001': '111110',
 '01110': '111111'}

## Evaluate entropy, total number of coded bits and avg. code word length along with how close we are to entropy.

In [133]:
# all compute
# let's now get an idea about number of bits
tot_bits = 0
entropy =0.
avg = 0
total_counts = 0.
codes = h.codes
for key in huffmap.keys():
    #pdb.set_trace()
    tot_bits += huffmap[key] * len(codes[key])
    total_counts += huffmap[key]
print("Total bits needed: {}".format(tot_bits))
for key in huffmap.keys():
    #pdb.set_trace()
    avg += huffmap[key] * len(codes[key])/total_counts
    p = float(huffmap[key])/total_counts
    entropy += -(p * np.log2(p))
print("Entropy: {}".format(entropy))
print("Avg code work length: {}".format(avg))
print("How close are we to Entropy :{} ".format(entropy/avg))
    

Total bits needed: 4219563
Entropy: 4.628487346668847
Avg code work length: 4.651193783068784
How close are we to Entropy :0.9951181487035452 


## End of useful stuff. Just test here

In [13]:
# bin has our desired output. It has shape
bin_out.shape
bin_out = (bin_out + 1)/2
bin_out = bin_out.int()
print(bin_out)
print(bin_out.shape)
np.unique(bin_out)

tensor([[[1, 0, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 0],
         ...,
         [0, 1, 1,  ..., 1, 1, 0],
         [1, 1, 1,  ..., 1, 0, 0],
         [1, 1, 0,  ..., 0, 1, 1]],

        [[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 0, 0],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0]],

        [[0, 1, 1,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 0, 0, 1],
         [1, 0, 0,  ..., 0, 1, 1],
         ...,
         [1, 0, 0,  ..., 0, 1, 1],
         [0, 1, 1,  ..., 0, 1, 0],
         [0, 0, 0,  ..., 1, 1, 1]],

        ...,

        [[0, 0, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [0, 1, 1,  ..., 0, 0, 0],
         [0, 1, 1,  ..., 0, 0, 0],
         [1, 1, 0,  ..., 1, 1, 0]],

        [[1, 1, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 1, 1],
         [0,

array([0, 1], dtype=int32)

## Now let's write this to binary residual and compress

In [71]:
h = HuffmanCoding()
residual_file = 'residual.txt'#
output_file = open(residual_file, "w")
bin_out = (bin_out + 1)//2
bin_out = bin_out.astype(int)
#pdb.set_trace()
for row in bin_out:
    #pdb.set_trace()
    np.savetxt(output_file, row)
h.compress(residual_file, '.')

{'1': 110415, '.': 215856, '0': 4424361, 'e': 215856, '+': 215856, ' ': 216205, '\n': 1450}
Compressed


In [52]:
bin_out = (bin_out + 1)//2

In [54]:
bin_out = bin_out.astype(int)

In [55]:
bin_out

array([[[1, 1, 1, ..., 1, 1, 0],
        [1, 0, 0, ..., 1, 0, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 0, 1, ..., 1, 0, 1],
        [1, 1, 0, ..., 1, 0, 0],
        [1, 1, 1, ..., 1, 0, 0]],

       [[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 0, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 0, 0],
        [1, 1, 1, ..., 1, 1, 0]],

       [[0, 1, 1, ..., 1, 1, 1],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 1, 0],
        ...,
        [1, 1, 0, ..., 0, 1, 0],
        [0, 0, 1, ..., 1, 1, 1],
        [0, 0, 1, ..., 1, 1, 1]],

       ...,

       [[1, 0, 0, ..., 1, 0, 0],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [0, 1, 1, ..., 1, 0, 0],
        [0, 1, 1, ..., 0, 0, 1],
        [1, 1, 1, ..., 0, 0, 0]],

       [[0, 1, 1, ..., 0, 0, 1],
        [0, 1, 0, ..., 1, 1, 0],
        [0, 1, 0, ..., 0, 0, 1],
        ...,
        [0, 1, 0, ..., 

In [66]:
h.codes

{' ': '0', '\n': '100', '0': '101', '1': '11'}

## Let's do a mixed approach here

In [29]:
h = HuffmanCoding()
residual_file = 'residual.txt'#
output_file = open(residual_file, "w")
chunk_size = 5
with torch.no_grad():
        for idx, sample in enumerate(dataset_train):
            model_input = sample['image']
            bin_out = enc(model_input).squeeze().numpy()
            bin_out = ((bin_out + 1)/2).astype(int)
            H,W,C = bin_out.shape
            #pdb.set_trace()
            j=k=l=0
            while j<H:
                k=0
                while k<W:
                    channel = np.split(bin_out[j][k][:],chunk_size)
                    for num in channel:
                        s = "" 
                        for ele in num:
                            s += str(ele)
                        #pdb.set_trace()
                        output_file.write(s + '\n')
                    k+=1
                j+=1
h.compress(residual_file, '.')

Compressed


In [30]:
h.codes

{'1': '0', '\n': '10', '0': '11'}